<a href="https://colab.research.google.com/github/Zorug/Steam-Stuff/blob/master/BS_%26_CSV_compare.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Info

comparação com os dados do site: https://www.steamcardexchange.net 

extração do csv: https://steam.tools/cards/

In [1]:
""" info da columns:
Game: Name of the set's game
# Cards: Number of cards in the set
Set Price: The total price of all the cards for the set
Card Avg: The averege price of a card in the set
Booster Avg: The average price of a card from the set's booster
Booster %: How many percent cheaper avarage booster cards are for the set
Emote Avg: The average price of the set's emoticons
BG Avg: The average price of the set's bachgrounds
Avg Qty: The average quantity of items on the market
Discount: The money you would make by selling all your drops for that game
Added: The date the set was added
"""

" info da columns:\nGame: Name of the set's game\n# Cards: Number of cards in the set\nSet Price: The total price of all the cards for the set\nCard Avg: The averege price of a card in the set\nBooster Avg: The average price of a card from the set's booster\nBooster %: How many percent cheaper avarage booster cards are for the set\nEmote Avg: The average price of the set's emoticons\nBG Avg: The average price of the set's bachgrounds\nAvg Qty: The average quantity of items on the market\nDiscount: The money you would make by selling all your drops for that game\nAdded: The date the set was added\n"

# Importanto libs e dados

In [2]:
import pandas as pd

#csv_source = 'STC_set_data.csv'
csv_source = 'https://raw.githubusercontent.com/Zorug/Steam-Stuff/master/STC_set_data.csv'
dados_csv = pd.read_csv(csv_source)
qnt_dados = dados_csv.shape[0]
qnt_dados

9156

In [3]:
dados_csv.head(2)

,Game,# Owned,# Unique,# Cards,Badge Lvl,Set Price,Price Diff,Card Avg,Booster Avg,Booster %,Emote Avg,BG Avg,Avg Qty,Discount,Added,AppId
0,City Game Studio,0,0,12,0,1.25,1.25,0.10,NaN,0.0,0.00,0.00,17,0.52,2019-10-24,726840
1,Transport Services,0,0,5,0,1.85,1.85,0.37,NaN,0.0,0.00,0.00,4,0.97,2019-10-23,993690


#Organizando DataSet

## Removendo dados inúteis para o teste

In [0]:
dados_csv = dados_csv.drop(['# Owned', '# Unique', 'Badge Lvl','Emote Avg', 'BG Avg', 'Discount', 'Added'], axis = 1)

In [5]:
dados_csv.sample(2)

,Game,# Cards,Set Price,Price Diff,Card Avg,Booster Avg,Booster %,Avg Qty,AppId
21,Die Young,10,0.70,0.70,0.07,0.08,-14.3,42,433170
1084,Rocket Valley Tycoon,10,0.81,0.81,0.08,0.10,-25.0,10,861920


##Reorganizando os tipos

In [6]:
dados_csv.dtypes

Game            object
# Cards          int64
Set Price       object
Price Diff      object
Card Avg       float64
Booster Avg    float64
Booster %       object
Avg Qty          int64
AppId            int64
dtype: object

In [0]:
def remove_virgula_perdida(sentenca): # aqui estamos esvitando um erro de conversão
    sentenca = sentenca.replace(',','')
    return sentenca

In [0]:
para_float = ['Set Price', 'Price Diff', 'Booster %']
for coluna in dados_csv[para_float]:
    dados_csv[coluna] = dados_csv[coluna].apply(remove_virgula_perdida)

dados_csv[para_float] = dados_csv[para_float].astype('float64')
dados_csv['AppId'] = dados_csv['AppId'].astype('str')

In [9]:
dados_csv.dtypes

Game            object
# Cards          int64
Set Price      float64
Price Diff     float64
Card Avg       float64
Booster Avg    float64
Booster %      float64
Avg Qty          int64
AppId           object
dtype: object

# BeautifulSoup

In [0]:
import bs4
from urllib.request import urlopen
from bs4 import BeautifulSoup

In [0]:
def arruma_string(string_to_go):
  if(string_to_go == 'A'): return 'nan'
  elif(string_to_go[0] == 'n'): return string_to_go[4:]
  else: return string_to_go

def captura_dados_cartas(soup):
  cards_script = soup.find("div",{"id":"cards"})
  card_list = cards_script.findAll("div",{"class":"showcase-element"})
  temporario = []
  for item in card_list: # retira itens vazios
    if not item.a == None: temporario.append(item)
  card_list = temporario

  precos_das_cartas = []
  for item in card_list: 
    try: 
      precos_das_cartas.append( arruma_string( ((item.find("div", {"class":"element-button"})).text)[8:] ) )
    except:
      print('Arumando erro 368020, sem valor para carta... adicinonando NaN')
      precos_das_cartas.append( 'nan' )

  precos_das_cartas = pd.Series(precos_das_cartas).apply(remove_virgula_perdida).astype('float64')

  return precos_das_cartas

In [0]:
def captura_dados_booster(soup):
  booster_script = soup.find("div",{"id":"booster"})

  try:
    sub_script = (booster_script.find("div",{"class":"showcase-element-container booster"}))
    booster_title = sub_script.img["alt"]
    preco_booster = arruma_string( ((sub_script.find("a", {"class":"button-blue"})).text)[8:] )
  except:
    print('Erro de booster inexistente.')
    booster_title = '------'
    preco_booster = 'nan'

  preco_booster = pd.Series(preco_booster).apply(remove_virgula_perdida).astype('float64').values[0]
  
  return booster_title, preco_booster

In [15]:
#for i in range(10):
for i in range(qnt_dados):
  #amostra = dados_csv.sample(1)
  amostra = dados_csv[dados_csv.index == i]
  AppId = amostra['AppId'].values[0]
  url = 'https://www.steamcardexchange.net/index.php?gamepage-appid-'+AppId

  uClient = urlopen(url)
  page_html = uClient.read()
  uClient.close()

  soup = BeautifulSoup(page_html, 'html.parser')

  precos_das_cartas = captura_dados_cartas(soup)
  booster_title, preco_booster = captura_dados_booster(soup)

  print(AppId)
  print('índice:',amostra['AppId'].index[0])
  print('Preços das cartas =', list(precos_das_cartas.values))
  print('Título:', booster_title, '\nPreço do booster:', preco_booster)
  print('')

726840
índice: 0
Preços das cartas = [0.05, 0.08, 0.05, 0.06, 0.05, 0.05, 0.05, 0.06, 0.06, 0.08, 0.07, 0.05]
Título: City Game Studio Booster-Pack 
Preço do booster: 0.14

993690
índice: 1
Preços das cartas = [0.07, 0.18, 0.08, 0.09, 0.18]
Título: Transport Services Booster-Pack 
Preço do booster: nan

740080
índice: 2
Preços das cartas = [0.11, 0.09, 0.08, 0.07, 0.08, 0.08, 0.05, 0.09, 0.05, 0.07]
Título: Deadly Days Booster-Pack 
Preço do booster: 0.21

1012840
índice: 3
Preços das cartas = [0.13, 0.11, 0.13, 0.11, 0.14, 0.16]
Título: Moons of Madness Booster-Pack 
Preço do booster: 0.23

1129250
índice: 4
Preços das cartas = [0.34, 0.13, 0.12, 0.15, 0.12, 0.15]
Título: NEKO-NIN exHeart 3 Booster-Pack 
Preço do booster: 0.47

359510
índice: 5
Preços das cartas = [0.15, 0.19, 0.14, 0.15, 0.23, 0.16, 0.4, 0.1, 0.08, 0.14, 0.11]
Título: Tangle Tower Booster-Pack 
Preço do booster: 0.65

954610
índice: 6
Preços das cartas = [0.24, 0.6, 0.34, 0.37, 0.24]
Título: Onii-chan Asobo Booster-P